In [1]:
import pandas as pd
import numpy as np

#读取数据
train_price = pd.read_csv("ruc_Class25Q2_train_rent.csv", low_memory=False)
test_price  = pd.read_csv("ruc_Class25Q2_test_rent.csv",  low_memory=False)

#查看数据
print("=====Train Price Data Info=====")
print(train_price.shape)
print(train_price.columns)
print(train_price.head(3))
print(train_price.info())

print("\n=====Test Price Data Info=====")
print(test_price.shape)
print(test_price.columns)
print(test_price.info())

# 检查缺失值
print("\n=====Missing Values in Train Price=====")
print(train_price.isnull().sum().sort_values(ascending=False).head(20))

=====Train Price Data Info=====
(98899, 46)
Index(['城市', '户型', '装修', 'Price', '楼层', '面积', '朝向', '交易时间', '付款方式', '租赁方式',
       '电梯', '车位', '用水', '用电', '燃气', '采暖', '租期', '配套设施', 'lon', 'lat', '年份',
       '区县', '板块', '环线位置', '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司',
       '绿 化 率', '容 积 率', '物 业 费', '建筑结构', '物业办公电话', '产权描述', '供水', '供暖', '供电',
       '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈'],
      dtype='object')
   城市      户型   装修          Price     楼层      面积 朝向        交易时间 付款方式 租赁方式  \
0   0  1室1厅1卫  精装修  654646.481811   4/6层  36.42㎡  西  2024-11-28  季付价   整租   
1   0  1室1厅1卫  精装修  665412.057415   4/6层  41.00㎡  南  2024-10-30  季付价   整租   
2   0  1室1厅1卫  精装修  778222.820548  1/18层  37.36㎡  北  2024-11-12  季付价   整租   

   ...     供水    供暖     供电            燃气费       供热费    停车位 停车费用     coord_x  \
0  ...     民水  集中供暖     民电       2.61元/m³  24-30元/㎡  450.0  150  117.339283   
1  ...     民水  集中供暖     民电       2.61元/m³     30元/㎡  150.0  150  117.446526   
2  ...  商水/民水  集

In [2]:
#处理数据

#填充板块的空值
from sklearn.neighbors import KNeighborsClassifier

def fill_block_by_location(df, block_col='板块', lon_col='lon', lat_col='lat', k=5):
    # 非空样本
    train_df = df[df[block_col].notna()]
    # 空值样本
    missing_df = df[df[block_col].isna()]

    if missing_df.empty:
        return df  # 没有空值

    # KNN 分类器
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(train_df[[lon_col, lat_col]], train_df[block_col])
    
    # 预测空值
    df.loc[df[block_col].isna(), block_col] = knn.predict(missing_df[[lon_col, lat_col]])
    
    return df

train_price = fill_block_by_location(train_price)
test_price = fill_block_by_location(test_price)

In [3]:
#户型
import re
def house_features(house_type):
    #空值直接填0室0厅0卫
    if pd.isna(house_type) or house_type.strip() == '':
        return pd.Series({'室': 0, '厅': 0, '卫': 0})
    
    text = str(house_type)
    
    #提取“室”“厅”“卫”的数字
    room = re.search(r'(\d+)\s*室', str(house_type))
    hall = re.search(r'(\d+)\s*厅', str(house_type))
    bath = re.search(r'(\d+)\s*卫', str(house_type))
    room_alt = re.search(r'(\d+)\s*房间', str(house_type))
    
    #提取数字或设默认值
    rooms = int(room.group(1)) if room else (int(room_alt.group(1)) if room_alt else np.nan)
    halls = int(hall.group(1)) if hall else 0
    baths = int(bath.group(1)) if bath else np.nan
    
    return pd.Series({'室': rooms, '厅': halls, '卫': baths})

for df in [train_price, test_price]:
    df[['室', '厅', '卫']] = df['户型'].apply(house_features)

#检查结果
print(train_price[['室', '厅', '卫']].head())
print(test_price[['室', '厅', '卫']].head())

     室    厅    卫
0  1.0  1.0  1.0
1  1.0  1.0  1.0
2  1.0  1.0  1.0
3  3.0  1.0  2.0
4  1.0  1.0  1.0
     室    厅    卫
0  2.0  2.0  1.0
1  2.0  1.0  1.0
2  2.0  2.0  1.0
3  2.0  1.0  1.0
4  3.0  2.0  2.0


In [4]:
#装修

#填充空值为'其他'
train_price['装修'] = train_price['装修'].fillna('其他')
test_price['装修'] = test_price['装修'].fillna('其他')

#生成哑变量
train_renov = pd.get_dummies(train_price['装修'], prefix='装修', drop_first=True)
test_renov = pd.get_dummies(test_price['装修'], prefix='装修', drop_first=True)

train_price = pd.concat([train_price, train_renov], axis=1)
test_price = pd.concat([test_price, test_renov], axis=1)

print(train_price.filter(like='装修_').head())

   装修_精装修
0    True
1    True
2    True
3    True
4    True


In [5]:
#所在楼层
def process_floor(df, col='楼层', region_col='板块'):
    # 定义提取函数
    def extract_floor_info(value):
        if pd.isna(value):
            return pd.Series([np.nan, np.nan, np.nan])
        
        val = str(value).replace("层", "").strip()
        
        # 1. 提取总楼层
        total_match = re.search(r'/(\d+)', val)
        total = int(total_match.group(1)) if total_match else np.nan
        
        # 2. 提取估计楼层
        num_match = re.match(r'(\d+)', val)
        if num_match:
            est_floor = int(num_match.group(1))
        elif "低" in val:
            est_floor = 1 if pd.notna(total) else np.nan
        elif "中" in val:
            est_floor = total // 2 if pd.notna(total) else np.nan
        elif "高" in val:
            est_floor = total - 1 if pd.notna(total) else np.nan
        elif "地下" in val or "负" in val:
            est_floor = 0
        else:
            est_floor = np.nan
        
        # 3. 实际楼层比
        rel_floor = est_floor / total if pd.notna(est_floor) and pd.notna(total) and total != 0 else np.nan
        
        return pd.Series([est_floor, total, rel_floor])
    
    # 应用提取函数
    df[['估计楼层', '总楼层', '实际楼层比']] = df[col].apply(extract_floor_info)
    
    # 按“板块”分组用中位数填补空值
    for c in ['估计楼层', '总楼层', '实际楼层比']:
        df[c] = df.groupby(region_col)[c].transform(lambda x: x.fillna(x.median()))
        df[c] = df[c].fillna(df[c].median())  # 如果板块也为空，用全局中位数
    
    return df

# 对 train_price 和 test_price 同时应用
train_price = process_floor(train_price)
test_price = process_floor(test_price)

# 检查结果
print(train_price[['楼层', '估计楼层', '总楼层', '实际楼层比']].head(10))
print(test_price[['楼层', '估计楼层', '总楼层', '实际楼层比']].head(10))

        楼层  估计楼层   总楼层     实际楼层比
0     4/6层   4.0   6.0  0.666667
1     4/6层   4.0   6.0  0.666667
2    1/18层   1.0  18.0  0.055556
3    1/10层   1.0  10.0  0.100000
4   18/18层  18.0  18.0  1.000000
5   17/17层  17.0  17.0  1.000000
6   17/17层  17.0  17.0  1.000000
7    5/17层   5.0  17.0  0.294118
8    5/17层   5.0  17.0  0.294118
9  中楼层/25层  12.0  25.0  0.480000
        楼层  估计楼层   总楼层     实际楼层比
0  低楼层/18层   1.0  18.0  0.055556
1   低楼层/8层   1.0   8.0  0.125000
2  中楼层/20层  10.0  20.0  0.500000
3  高楼层/12层  11.0  12.0  0.916667
4  中楼层/23层  11.0  23.0  0.478261
5   30/45层  30.0  45.0  0.666667
6  中楼层/34层  17.0  34.0  0.500000
7   中楼层/6层   3.0   6.0  0.500000
8  高楼层/32层  31.0  32.0  0.968750
9   中楼层/9层   4.0   9.0  0.444444


In [6]:
#面积

def extract_area(area_str):
    if pd.isna(area_str):
        return None
    # 提取数字部分，例如 "33㎡" -> 33
    match = re.search(r"[\d.]+", str(area_str))
    return float(match.group()) if match else None

# 对 train_price 和 test_price 分别处理
for df in [train_price, test_price]:
    df["面积_处理后"] = df["面积"].apply(extract_area)

#检查结果
print(train_price[['面积_处理后']].head())
print(test_price[['面积_处理后']].head())

   面积_处理后
0   36.42
1   41.00
2   37.36
3   55.42
4   49.30
   面积_处理后
0   86.94
1   72.60
2   98.00
3   98.97
4  170.53


In [7]:
#房屋朝向
directions = ['东', '南', '西', '北', '东南', '东北', '西南', '西北']

def process_orientation(df, col='朝向'):
    for dir_ in directions:
        #有这个方向标1，否则0
        df[f'朝向_{dir_}'] = df[col].apply(lambda x: 1 if pd.notna(x) and dir_ in x else 0)
    return df

train_price = process_orientation(train_price)
test_price = process_orientation(test_price)

#检查结果
print(train_price[[f'朝向_{d}' for d in directions]].head())

   朝向_东  朝向_南  朝向_西  朝向_北  朝向_东南  朝向_东北  朝向_西南  朝向_西北
0     0     0     1     0      0      0      0      0
1     0     1     0     0      0      0      0      0
2     0     0     0     1      0      0      0      0
3     0     1     0     0      0      0      0      0
4     0     1     0     0      0      0      0      0


In [8]:
#交易时间
def process_trade_dates(df):
    # 统一时间格式
    df['交易时间'] = pd.to_datetime(df['交易时间'], errors='coerce')
    df['交易年份'] = df['交易时间'].dt.year

    return df

train_price = process_trade_dates(train_price)
test_price = process_trade_dates(test_price)

#检查结果
print(train_price[['交易年份']].head())
print(test_price[['交易年份']].head())

   交易年份
0  2024
1  2024
2  2024
3  2024
4  2024
   交易年份
0  2025
1  2025
2  2025
3  2025
4  2025


In [9]:
# 付款方式
# 定义付款方式关键词
patterns = {
    "季付价": "季付",
    "月付价": "月付",
    "半年付价": "半年付"
}

# 对 train_price 和 test_price 分别处理
for df in [train_price, test_price]:
    # 遍历每个模式，创建哑变量列（True/False）
    for new_col, keyword in patterns.items():
        df[new_col] = df["付款方式"].astype(str).str.contains(keyword).fillna(False)
    
    # 如果都不是这三种，则全部为 False

# 检查结果
print(train_price[["付款方式", "季付价", "月付价", "半年付价"]].head())

  付款方式   季付价    月付价   半年付价
0  季付价  True  False  False
1  季付价  True  False  False
2  季付价  True  False  False
3  季付价  True  False  False
4  季付价  True  False  False


In [10]:
# 租赁方式
# 对 train_price 和 test_price 分别处理
for df in [train_price, test_price]:
    df['整租'] = df['租赁方式'].apply(lambda x: True if x == '整租' else False)

# 检查结果
print(train_price[['租赁方式', '整租']].head())

  租赁方式    整租
0   整租  True
1   整租  True
2   整租  True
3   整租  True
4   整租  True


In [11]:
# 电梯
for df in [train_price, test_price]:
    df['有无电梯'] = df['电梯'].apply(lambda x: True if x == '有' else False)

# 检查结果
print(train_price[['电梯', '有无电梯']].head())

  电梯   有无电梯
0  无  False
1  无  False
2  有   True
3  有   True
4  有   True


In [12]:
# 车位
for df in [train_price, test_price]:
    df['车位_租用'] = df['车位'].apply(lambda x: True if x == '租用车位' else False)
    df['车位_免费'] = df['车位'].apply(lambda x: True if x == '免费使用' else False)

# 检查结果
print(train_price[["车位_租用", "车位_免费"]].head())

   车位_租用  车位_免费
0  False  False
1  False  False
2   True  False
3   True  False
4  False  False


In [13]:
# 水电
# 对 train_price 和 test_price 分别处理
for df in [train_price, test_price]:
    df['民用水电'] = df['用水'].apply(lambda x: True if x == '民水' else False)

# 检查结果
print(train_price[['民用水电', '用水']].head())

    民用水电  用水
0   True  民水
1   True  民水
2   True  民水
3  False  商水
4   True  民水


In [14]:
# 燃气
for df in [train_price, test_price]:
    df['有无燃气'] = df['燃气'].apply(lambda x: True if x == '有' else False)

# 检查结果
print(train_price[['燃气', '有无燃气']].head())

  燃气   有无燃气
0  有   True
1  有   True
2  有   True
3  无  False
4  有   True


In [15]:
#供暖
def process_heating(df, col='采暖', region_col='板块'):
    # 根据板块众数填充空值
    df[col] = df.groupby(region_col)[col].transform(
        lambda x: x.fillna(x.mode()[0] if not x.mode().empty else '自采暖')
    )
    # 生成哑变量：集中供暖为 True，其它为 False
    df['集中供暖'] = df[col].apply(lambda x: True if x == '集中供暖' else False)
    return df

train_price = process_heating(train_price)
test_price = process_heating(test_price)

# 检查
print(train_price[['采暖','集中供暖']].head(10))

     采暖   集中供暖
0  集中供暖   True
1  集中供暖   True
2  集中供暖   True
3  集中供暖   True
4  集中供暖   True
5  集中供暖   True
6  集中供暖   True
7  集中供暖   True
8  集中供暖   True
9   自采暖  False


In [16]:
#租期
def process_rent_period(df, col='租期', fill_by='板块'):
    """
    将租期转换为数值（月），空值按板块均值填充
    """
    def parse_period(x):
        if pd.isna(x):
            return np.nan
        x = str(x).replace(' ', '')  # 去掉空格
        # 匹配 "5~12个月"
        match = re.match(r'(\d+)~(\d+)个月', x)
        if match:
            return (int(match.group(1)) + int(match.group(2))) / 2
        # 匹配 "1年"、"2年以内"、"1~2年"
        match_year_range = re.match(r'(\d+)~(\d+)年', x)
        if match_year_range:
            return ((int(match_year_range.group(1)) + int(match_year_range.group(2))) / 2) * 12
        match_year = re.match(r'(\d+)年', x)
        if match_year:
            return int(match_year.group(1)) * 12
        # 匹配 "X年以上" -> 取 X+1 年作为大致值
        match_more = re.match(r'(\d+)年以上', x)
        if match_more:
            return (int(match_more.group(1)) + 1) * 12
        # 匹配 "X个月以内" -> 取 X/2 作为大致值
        match_less = re.match(r'(\d+)个月以内', x)
        if match_less:
            return int(match_less.group(1)) / 2
        return np.nan  # 其它情况返回 NaN

    # 转换为数值
    df[col + '_处理后'] = df[col].apply(parse_period)

    # 按板块均值填充空值
    df[col + '_处理后'] = df.groupby(fill_by)[col + '_处理后'].transform(
        lambda x: x.fillna(x.mean())
    )

    # 如果还有空值，用全局均值填充
    df[col + '_处理后'] = df[col + '_处理后'].fillna(df[col + '_处理后'].mean())
    
    return df

train_price = process_rent_period(train_price)
test_price = process_rent_period(test_price)

# 检查
print(train_price[['租期','租期_处理后']].head(10))

       租期     租期_处理后
0      1年  12.000000
1     NaN  30.486486
2     NaN  27.120915
3  5~12个月   8.500000
4      1年  12.000000
5    1~2年  18.000000
6    1~2年  18.000000
7      1年  12.000000
8      1年  12.000000
9    3年以内  36.000000


In [17]:
#配套
def process_facilities(df, col='配套设施'):
    """
    将配套设施生成哑变量
    """
    # 选择要生成的主要设施
    facilities = ['洗衣机','空调','电视','冰箱','热水器','床','宽带']
    
    # 初始化列
    for f in facilities:
        df[f] = df[col].apply(lambda x: True if f in str(x) else False)
    
    return df

# 应用于训练集和测试集
train_price = process_facilities(train_price)
test_price = process_facilities(test_price)

# 检查
print(train_price[['配套设施'] + ['洗衣机','空调','电视','冰箱','热水器','床','宽带']].head(10))

                           配套设施   洗衣机    空调     电视     冰箱   热水器     床     宽带
0            洗衣机、空调、衣柜、热水器、床、宽带  True  True  False  False  True  True   True
1         洗衣机、空调、衣柜、电视、热水器、床、宽带  True  True   True  False  True  True   True
2         洗衣机、空调、衣柜、电视、热水器、床、宽带  True  True   True  False  True  True   True
3   洗衣机、空调、衣柜、电视、冰箱、热水器、床、暖气、宽带  True  True   True   True  True  True   True
4         洗衣机、空调、衣柜、电视、热水器、床、宽带  True  True   True  False  True  True   True
5            洗衣机、空调、衣柜、热水器、床、宽带  True  True  False  False  True  True   True
6            洗衣机、空调、衣柜、热水器、床、宽带  True  True  False  False  True  True   True
7               洗衣机、空调、热水器、床、宽带  True  True  False  False  True  True   True
8               洗衣机、空调、热水器、床、宽带  True  True  False  False  True  True   True
9  洗衣机、空调、衣柜、电视、冰箱、热水器、床、暖气、天然气  True  True   True   True  True  True  False


In [18]:
#建筑年代
def process_building_year(df, col='建筑年代', region_col='板块'):
    """
    格式如 '1955-2000年' 或 '2000年'
    提取平均年份
    缺失值用所在板块的中位数填充
    """
    def extract_avg_year(text):
        if pd.isna(text):
            return np.nan
        # 提取所有4位数字年份
        years = re.findall(r'\d{4}', str(text))
        if len(years) == 2:
            return (float(years[0]) + float(years[1])) / 2
        elif len(years) == 1:
            return float(years[0])
        else:
            return np.nan

    # 转换为数值列
    df['平均建筑年代'] = df[col].apply(extract_avg_year)

    # 按板块计算中位数（这里用 numeric 列，而不是原始 object 列）
    median_by_region = df.groupby(region_col)['平均建筑年代'].transform('median')
    df['平均建筑年代'] = df['平均建筑年代'].fillna(median_by_region)

    # 若板块中也都是空，再用全局中位数
    df['平均建筑年代'] = df['平均建筑年代'].fillna(df['平均建筑年代'].median())

    return df

# 应用
train_price = process_building_year(train_price)
test_price = process_building_year(test_price)

print(train_price[['建筑年代', '平均建筑年代']].head(10))

         建筑年代  平均建筑年代
0  1963-2001年  1982.0
1  1988-2002年  1995.0
2  2004-2009年  2006.5
3  2015-2018年  2016.5
4  1980-1996年  1988.0
5  1985-1993年  1989.0
6  1997-1998年  1997.5
7  1985-1993年  1989.0
8  1997-1998年  1997.5
9  2002-2006年  2004.0


In [19]:
#房屋总数
def process_total_households(df, col='房屋总数', group_col='板块'):
    new_col = col + '_处理后'
    
    # 去掉单位并提取数字
    df[new_col] = (
        df[col]
        .astype(str)
        .str.replace('户', '', regex=False)
        .str.extract(r'(\d+)')
        .astype(float)
    )

    # 按板块中位数填补空值
    median_by_group = df.groupby(group_col)[new_col].transform('median')
    df[new_col] = df[new_col].fillna(median_by_group)

    # 若板块中也全空，再用全局中位数
    df[new_col] = df[new_col].fillna(df[new_col].median())

    return df

# 应用到训练集和测试集
train_price = process_total_households(train_price, '房屋总数', '板块')
test_price = process_total_households(test_price, '房屋总数', '板块')

# 查看结果
print(train_price[['房屋总数', '房屋总数_处理后']].head(10))

    房屋总数  房屋总数_处理后
0  1731户    1731.0
1  1931户    1931.0
2  1891户    1891.0
3  3026户    3026.0
4  3031户    3031.0
5  1018户    1018.0
6   546户     546.0
7  1018户    1018.0
8   546户     546.0
9   458户     458.0


In [20]:
#绿化率
def process_total_households(df, col='绿 化 率', group_col='板块'):
    new_col = col + '_处理后'
    
    # 去掉单位并提取数字
    df[new_col] = (
        df[col]
        .astype(str)
        .str.replace('%', '', regex=False)
        .str.extract(r'(\d+)')
        .astype(float)
    )

    # 按板块中位数填补空值
    median_by_group = df.groupby(group_col)[new_col].transform('median')
    df[new_col] = df[new_col].fillna(median_by_group)

    # 若板块中也全空，再用全局中位数
    df[new_col] = df[new_col].fillna(df[new_col].median())

    return df

# 应用到训练集和测试集
train_price = process_total_households(train_price,'绿 化 率', '板块')
test_price = process_total_households(test_price, '绿 化 率', '板块')

# 查看结果
print(train_price[['绿 化 率', '绿 化 率_处理后']].head(10))

    绿 化 率  绿 化 率_处理后
0     30%       30.0
1     30%       30.0
2     30%       30.0
3     35%       35.0
4     25%       25.0
5     10%       10.0
6     30%       30.0
7     10%       10.0
8     30%       30.0
9  14.99%       14.0


In [21]:
# 容积率
def process_floor_area_ratio(df, col='容 积 率', region_col='板块'):
    # 按区域中位数填充缺失值
    median_by_region = df.groupby(region_col)[col].transform('median')
    df[col + '_处理后'] = df[col].fillna(median_by_region)

    # 区域中也全空的用全局中位数
    df[col + '_处理后'] = df[col + '_处理后'].fillna(df[col + '_处理后'].median())

    return df

train_price = process_floor_area_ratio(train_price)
test_price = process_floor_area_ratio(test_price)

print(train_price[['容 积 率', '容 积 率_处理后']].head(10))

   容 积 率  容 积 率_处理后
0   2.50       2.50
1   1.20       1.20
2   2.70       2.70
3   2.80       2.80
4   1.70       1.70
5   2.00       2.00
6   3.15       3.15
7   2.00       2.00
8   3.15       3.15
9   4.30       4.30


In [22]:
def process_property_fee(df, col='物 业 费', region_col='板块'):
    """
    处理物业费列：
    格式如 '1-1.63元/月/㎡' 或空-提取平均值
    缺失值用所在区域的中位数填充，如果区域全空再用全局中位数
    """
    def extract_mid(s):
        if pd.isna(s):
            return np.nan
        # 提取数字
        nums = re.findall(r'[\d\.]+', str(s))
        nums = [float(x) for x in nums]
        if len(nums) == 1:
            return nums[0]
        elif len(nums) >= 2:
            return np.mean(nums[:2])
        else:
            return np.nan
    
    # 提取物业费数值
    df[col+'_num'] = df[col].map(extract_mid)

    # 按区域中位数填充
    median_by_region = df.groupby(region_col)[col+'_num'].transform('median')
    df[col+'_num'] = df[col+'_num'].fillna(median_by_region)

    # 区域中也全空的用全局中位数
    df[col+'_num'] = df[col+'_num'].fillna(df[col+'_num'].median())

    return df

# 应用
train_price = process_property_fee(train_price)
test_price = process_property_fee(test_price)

print(train_price[['物 业 费', '物 业 费_num']].head(10))

           物 业 费  物 业 费_num
0  1.1-1.85元/月/㎡      1.475
1  0.6-1.15元/月/㎡      0.875
2    2-2.66元/月/㎡      2.330
3   1.5-4.9元/月/㎡      3.200
4   0.8-1.2元/月/㎡      1.000
5       0.8元/月/㎡      0.800
6      1.75元/月/㎡      1.750
7       0.8元/月/㎡      0.800
8      1.75元/月/㎡      1.750
9       2.8元/月/㎡      2.800


In [23]:
#建筑结构
def process_building_type(df, col='建筑结构', group_col='板块'):
    types = ['塔楼', '板楼', '塔板结合', '平房']
    
    # 1. 空值先填空字符串
    df[col] = df[col].fillna('')
    
    # 2. 遍历每个类型生成哑变量
    for t in types:
        df[f'结构_{t}'] = df[col].apply(lambda x: t in x)
    
    # 3. 针对四个都为 False 的行填充同板块中出现次数最多的类型
    dummy_cols = [f'结构_{t}' for t in types]
    
    mask_all_false = df[dummy_cols].sum(axis=1) == 0
    
    # 按板块统计每种类型数量最多，并转换为 bool
    mode_by_group = df.groupby(group_col)[dummy_cols].transform(lambda x: x.mode().iloc[0].astype(bool) if not x.mode().empty else x.iloc[0])
    
    # 填充全为 False 的行
    df.loc[mask_all_false, dummy_cols] = mode_by_group.loc[mask_all_false].astype(bool)
    
    return df

train_price = process_building_type(train_price, col='建筑结构')
test_price = process_building_type(test_price, col='建筑结构')

print(train_price[['建筑结构'] + [f'结构_{t}' for t in ['塔楼', '板楼','塔板结合','平房']]].head(10))

         建筑结构  结构_塔楼  结构_板楼  结构_塔板结合  结构_平房
0       塔楼/板楼   True   True    False  False
1  塔楼/板楼/塔板结合   True   True     True  False
2  塔楼/板楼/塔板结合   True   True     True  False
3  塔楼/板楼/塔板结合   True   True     True  False
4       塔楼/板楼   True   True    False  False
5          塔楼   True  False    False  False
6          板楼  False   True    False  False
7          塔楼   True  False    False  False
8          板楼  False   True    False  False
9     塔楼/塔板结合   True  False     True  False


In [24]:
#产权描述
def process_property_type(df, col='产权描述'):
    """
    根据产权描述列生成三个哑变量：
    - 产权_高价：完全产权（商品房）
    - 产权_中价：部分限制（已购公房、央产房、共有产权房等）
    - 产权_低价：限制明显（经济适用房、私产、自建、小产权等）
    若某行无法识别（空白或其他），默认为中价档
    """
    df[col] = df[col].astype(str).fillna('')

    # 高价档：完全产权
    high_pattern = r'商品房'

    # 中价档：部分限制（已购公房、央产、共有产权）
    mid_pattern = r'已购公房|央产房|共有|使用权'

    # 低价档：经济适用、私产、自建、小产权
    low_pattern = r'经济适用|私产|自建|小产'

    # 生成哑变量（用str.contains匹配所有可能分隔形式）
    df['产权_高价'] = df[col].str.contains(high_pattern, regex=True)
    df['产权_中价'] = df[col].str.contains(mid_pattern, regex=True)
    df['产权_低价'] = df[col].str.contains(low_pattern, regex=True)

    # 若全部为False（无法识别或空白），默认为高价（完全产权）
    none_idx = ~(df[['产权_高价', '产权_中价', '产权_低价']].any(axis=1))
    df.loc[none_idx, '产权_中价'] = True

    # 转换为int类型
    df[['产权_高价', '产权_中价', '产权_低价']] = df[['产权_高价', '产权_中价', '产权_低价']].astype(int)

    return df

# 应用到训练集和测试集
train_price = process_property_type(train_price, '产权描述')
test_price = process_property_type(test_price, '产权描述')

# 查看结果
print(train_price[['产权描述','产权_高价','产权_中价','产权_低价']].head(10))

                          产权描述  产权_高价  产权_中价  产权_低价
0      商品房/已购公房/央产房/二类经济适用房/私产      1      1      1
1              商品房/已购公房/使用权/私产      1      1      1
2                       商品房/私产      1      0      1
3                   商品房/自住型商品房      1      0      0
4      商品房/已购公房/央产房/二类经济适用房/私产      1      1      1
5      商品房/已购公房/央产房/二类经济适用房/私产      1      1      1
6  商品房/已购公房/一类经济适用房/二类经济适用房/私产      1      1      1
7      商品房/已购公房/央产房/二类经济适用房/私产      1      1      1
8  商品房/已购公房/一类经济适用房/二类经济适用房/私产      1      1      1
9                       商品房/私产      1      0      1


In [25]:
#燃气费、供热费
def process_fee(df, col, group_col='板块'):
    def str_to_mean(x):
        if pd.isna(x):
            return np.nan  # 先保留空值为 np.nan，方便后续按板块均值填充
        # 提取数字
        nums = re.findall(r'[\d.]+', str(x))
        if len(nums) == 0:
            return np.nan
        nums = [float(n) for n in nums]
        return np.mean(nums)

    # 1. 先提取数值
    df[col] = df[col].apply(str_to_mean)

    # 2. 按板块均值填充空值
    df[col] = df[col].fillna(df.groupby(group_col)[col].transform('mean'))

    # 3. 若板块中全部都是空，再用全局均值
    df[col] = df[col].fillna(df[col].mean())

    return df

# 调用
train_price = process_fee(train_price, '燃气费', '板块')
train_price = process_fee(train_price, '供热费', '板块')
test_price = process_fee(test_price, '燃气费', '板块')
test_price = process_fee(test_price, '供热费', '板块')

print(train_price[['燃气费','供热费']].head(10))

    燃气费   供热费
0  2.61  27.0
1  2.61  30.0
2  2.62  38.0
3  2.61  37.0
4  2.61  30.0
5  2.61  30.0
6  2.61  24.0
7  2.61  30.0
8  2.61  24.0
9  2.61  30.0


In [26]:
#停车位
def process_parking_spots(df, col='停车位', region_col='板块'):
    # 转换为数值类型，非数值的设为 NaN
    df[col + '_处理后'] = pd.to_numeric(df[col], errors='coerce')

    # 按区域中位数填充缺失值
    median_by_region = df.groupby(region_col)[col + '_处理后'].transform('median')
    df[col + '_处理后'] = df[col + '_处理后'].fillna(median_by_region)

    # 区域中也全空的用全局中位数
    df[col + '_处理后'] = df[col + '_处理后'].fillna(df[col + '_处理后'].median())

    return df

train_price = process_parking_spots(train_price)
test_price = process_parking_spots(test_price)

print(train_price[['停车位', '停车位_处理后']].head(10))

     停车位  停车位_处理后
0  450.0    450.0
1  150.0    150.0
2  965.0    965.0
3  500.0    500.0
4  400.0    400.0
5    NaN    650.0
6  150.0    150.0
7    NaN    650.0
8  150.0    150.0
9  280.0    280.0


In [27]:
#停车费用
def process_parking_fee(df, col='停车费用', region_col='板块'):
    """
    处理停车费用列：
    - '暂无' -> 0
    - '地上150元/月/位，地下500元/月/位' -> 提取第一个数字 (150)
    - 其他非数字描述 -> 0
    - 缺失值用所在区域中位数填充
    - 区域中全空的用全局中位数
    """
    def extract_fee(val):
        if pd.isna(val):
            return np.nan  # 先用 NaN
        val_str = str(val)
        if '暂无' in val_str:
            return np.nan
        # 尝试提取第一个数字
        match = re.search(r'(\d+\.?\d*)', val_str)
        if match:
            return float(match.group(1))
        return np.nan  # 非数字的先设为 NaN

    # 提取停车费用
    df['停车费用_处理后'] = df[col].apply(extract_fee)

    # 按区域中位数填充缺失值
    median_by_region = df.groupby(region_col)['停车费用_处理后'].transform('median')
    df['停车费用_处理后'] = df['停车费用_处理后'].fillna(median_by_region)

    # 区域中也全空的用全局中位数
    df['停车费用_处理后'] = df['停车费用_处理后'].fillna(df['停车费用_处理后'].median())

    return df

# 应用
train_price = process_parking_fee(train_price)
test_price = process_parking_fee(test_price)

# 检查结果
print(train_price[['停车费用', '停车费用_处理后']].head(10))

                                 停车费用  停车费用_处理后
0                                 150     150.0
1                                 150     150.0
2                                 500     500.0
3                                 550     550.0
4                                 150     150.0
5                                 150     150.0
6                                 120     120.0
7                                 150     150.0
8                                 120     120.0
9  地上150元/月/位，地下2元/时/位，地下固定车位450元/月/位     150.0


In [28]:
#坐标
def process_city_and_coord(df):
    df = df.copy()
    
    #坐标二次项
    if 'coord_x' in df.columns and 'coord_y' in df.columns:
        x = df['coord_x']
        y = df['coord_y']
        # 二次项
        df['coord_x2'] = x**2
        df['coord_y2'] = y**2
        df['coord_xy'] = x*y
    
    #环线处理
    if '环线位置' in df.columns and '城市' in df.columns:
        # 环线哑变量
        ring_dummies = pd.get_dummies(df['环线位置'], prefix='环线', drop_first=True)
        df = pd.concat([df, ring_dummies], axis=1)
    
    #城市哑变量
    if '城市' in df.columns:
        city_dummies = pd.get_dummies(df['城市'], prefix='城市', drop_first=True)
        df = pd.concat([df, city_dummies], axis=1)
    
    return df

# === 应用到训练集和测试集 ===
train_price = process_city_and_coord(train_price)
test_price = process_city_and_coord(test_price)

# === 检查处理结果 ===
cols_to_check = [c for c in train_price.columns if ('环线_' in c) or ('城市_' in c) or ('coord' in c)]
print(train_price[cols_to_check].head())

      coord_x    coord_y      coord_x2     coord_y2     coord_xy  环线_中环至外环  \
0  117.339283  40.930007  13768.507300  1675.265468  4802.697662     False   
1  117.446526  40.876743  13793.686376  1670.908082  4800.831391     False   
2  117.518524  40.905357  13810.603462  1673.248219  4807.137157     False   
3  117.672869  41.121247  13846.904118  1690.956995  4838.855172     False   
4  117.363538  40.991043  13774.199978  1680.265591  4810.853798     False   

   环线_二环内  环线_二至三环  环线_五至六环  环线_六环外  ...   城市_2   城市_3   城市_4   城市_5   城市_6  \
0   False    False    False   False  ...  False  False  False  False  False   
1   False     True    False   False  ...  False  False  False  False  False   
2   False    False    False   False  ...  False  False  False  False  False   
3   False    False    False    True  ...  False  False  False  False  False   
4   False    False    False   False  ...  False  False  False  False  False   

    城市_7   城市_8   城市_9  城市_10  城市_11  
0  False  False  

In [29]:
#再次查看数据
print("=====Train Price Data Info=====")
print(train_price.columns.tolist())

=====Train Price Data Info=====
['城市', '户型', '装修', 'Price', '楼层', '面积', '朝向', '交易时间', '付款方式', '租赁方式', '电梯', '车位', '用水', '用电', '燃气', '采暖', '租期', '配套设施', 'lon', 'lat', '年份', '区县', '板块', '环线位置', '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司', '绿 化 率', '容 积 率', '物 业 费', '建筑结构', '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室', '厅', '卫', '装修_精装修', '估计楼层', '总楼层', '实际楼层比', '面积_处理后', '朝向_东', '朝向_南', '朝向_西', '朝向_北', '朝向_东南', '朝向_东北', '朝向_西南', '朝向_西北', '交易年份', '季付价', '月付价', '半年付价', '整租', '有无电梯', '车位_租用', '车位_免费', '民用水电', '有无燃气', '集中供暖', '租期_处理后', '洗衣机', '空调', '电视', '冰箱', '热水器', '床', '宽带', '平均建筑年代', '房屋总数_处理后', '绿 化 率_处理后', '容 积 率_处理后', '物 业 费_num', '结构_塔楼', '结构_板楼', '结构_塔板结合', '结构_平房', '产权_高价', '产权_中价', '产权_低价', '停车位_处理后', '停车费用_处理后', 'coord_x2', 'coord_y2', 'coord_xy', '环线_中环至外环', '环线_二环内', '环线_二至三环', '环线_五至六环', '环线_六环外', '环线_内环内', '环线_内环至中环', '环线_内环至外环', '环线_四至五环', '环线_外环外', '城市_1', '城市_2', '城市_3', '城市_4', '城市_5', '城市_6', '城市_7', '城市_8', '城市_9',

In [30]:
#处理异常值

#数值特征列表
num_features = [
    'coord_x', 'coord_y', 'coord_x2', 'coord_y2', 'coord_xy', '年份', '燃气费', '供热费', 
    '室', '厅', '卫', 
    '估计楼层', '总楼层', '实际楼层比', 
    '面积_处理后', '交易年份', 
    '租期_处理后', 
    '平均建筑年代', '房屋总数_处理后', 
    '绿 化 率_处理后', '容 积 率_处理后', '物 业 费_num', 
    '停车位_处理后', '停车费用_处理后'
]

#哑变量列表
dummy_features = [
    '装修_精装修', 
    '朝向_东', '朝向_南', '朝向_西', '朝向_北', 
    '朝向_东南', '朝向_东北', '朝向_西南', '朝向_西北', 
    '季付价', '月付价', '半年付价', '整租', '有无电梯', 
    '车位_租用', '车位_免费', '民用水电', '有无燃气', 
    '集中供暖', 
    '洗衣机', '空调', '电视', '冰箱', '热水器', '床', '宽带', 
    '环线_中环至外环', '环线_二环内', '环线_二至三环', '环线_五至六环', 
    '环线_六环外', '环线_内环内', '环线_内环至中环', '环线_内环至外环', 
    '环线_四至五环', '环线_外环外', 
    '结构_塔楼', '结构_板楼', '结构_塔板结合', '结构_平房', 
    '产权_高价', '产权_中价', '产权_低价', '城市_1', '城市_2', '城市_3', '城市_4', '城市_5', '城市_6', '城市_7', '城市_8', '城市_9', '城市_10', '城市_11'
]

others = ['板块', 'Price','ID']

#去除所有object列（除板块、价格）
keep_cols = num_features + dummy_features + others
train_price = train_price[train_price.columns.intersection(keep_cols)]
test_price = test_price[test_price.columns.intersection(keep_cols)]

def clean_numeric_outliers_by_region(df, numeric_cols, region_col='板块', method='iqr', factor=1.5):
    """
    按区域清理数值列异常值，并用区域中位数填充，若区域无有效值则用全局中位数。
    """
    df = df.copy()
    
    for col in numeric_cols:
        if method == 'iqr':
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower = Q1 - factor * IQR
            upper = Q3 + factor * IQR
        elif method == 'zscore':
            mean = df[col].mean()
            std = df[col].std()
            lower = mean - 3*std
            upper = mean + 3*std
        
        # 将异常值先设为 NaN
        df[col] = df[col].mask((df[col] < lower) | (df[col] > upper), np.nan)
        
        # 按区域填充中位数
        df[col] = df.groupby(region_col)[col].transform(
            lambda x: x.fillna(x.median())
        )
        
        # 对仍然缺失的值，用全局中位数填充
        global_median = df[col].median()
        df[col] = df[col].fillna(global_median)
    
    return df

train_price = clean_numeric_outliers_by_region(train_price, num_features, region_col='板块')
test_price = clean_numeric_outliers_by_region(test_price, num_features, region_col='板块')

train_price = train_price.drop(columns='板块')
test_price = test_price.drop(columns='板块')

#检查
# 把所有 bool 列转成 int
bool_cols = train_price.select_dtypes(include='bool').columns
train_price[bool_cols] = train_price[bool_cols].astype(int)
test_price[bool_cols] = test_price[bool_cols].astype(int)

# 找出 train 中不是 float 或 int 的列
non_numeric_train = train_price.select_dtypes(exclude=['float64', 'int64', 'int32']).columns
print("Train 非数值列:", list(non_numeric_train))

# 找出 test 中不是 float 或 int 的列
non_numeric_test = test_price.select_dtypes(exclude=['float64', 'int64', 'int32']).columns
print("Test 非数值列:", list(non_numeric_test))

Train 非数值列: []
Test 非数值列: []


In [31]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

scaler = StandardScaler()

# y 是目标变量，X 是特征变量
y = np.log(train_price['Price'])
X = train_price.drop(columns=['Price'])

#划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=111
)

#标准化
X_train[num_features] = scaler.fit_transform(X_train[num_features])
X_test[num_features] = scaler.transform(X_test[num_features])

#加上常数项
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

In [32]:
#OLS回归
model_OLS = sm.OLS(y_train, X_train).fit()

#查看回归结果
print(model_OLS.summary())

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

#验证
y_pred = model_OLS.predict(X_test)

from sklearn.metrics import r2_score

r2_val = r2_score(y_test, y_pred)
print("验证集 R²:", r2_val)

#交叉验证
model = LinearRegression()
scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
print("交叉验证 R²:", np.mean(scores))

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.764
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     3366.
Date:                Wed, 29 Oct 2025   Prob (F-statistic):               0.00
Time:                        17:41:58   Log-Likelihood:                -33395.
No. Observations:               79119   AIC:                         6.694e+04
Df Residuals:                   79042   BIC:                         6.766e+04
Df Model:                          76                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.4597      0.115    117.125      0.0

In [34]:
#Lasso
from sklearn.linear_model import Lasso, LassoCV
from sklearn.metrics import mean_squared_error, r2_score

# 使用交叉验证确定最优 alpha
lasso_cv = LassoCV(
    alphas=np.logspace(-4, 2, 50),
    cv=5,
    random_state=42,
    max_iter=10000
)
lasso_cv.fit(X_train, y_train)

best_alpha = lasso_cv.alpha_
print(f"✅ 最优 alpha: {best_alpha:.6f}")

# 用最优 alpha 重新训练 Lasso 模型
lasso_model = Lasso(alpha=best_alpha, max_iter=10000, random_state=42)
lasso_model.fit(X_train, y_train)

# R² in
y_pred_log_in = lasso_model.predict(X_train)
r2_log_in = r2_score(y_train, y_pred_log_in)
print(f"R²:in model: {r2_log_in:.6f}")

# 验证集预测 + R²（在 log 空间计算）
y_pred_log_out = lasso_model.predict(X_test)
r2_log_out = r2_score(y_test, y_pred_log_out)
print(f"R²:out of model: {r2_log_out:.6f}")

# 查看非零特征
coef_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': lasso_model.coef_
})
important_features = coef_df[coef_df['Coefficient'] != 0].sort_values(by='Coefficient', ascending=False)

print("\n📊 非零特征数量:", len(important_features))
print(important_features.head(10))

#交叉验证
# 交叉验证的每折 MSE 路径：shape = (n_alphas, n_folds)
mse_path = lasso_cv.mse_path_  

# 平均 MSE
mean_mse = mse_path.mean(axis=1)

# 对应最优 alpha 的平均 MSE
best_alpha_idx = np.where(lasso_cv.alphas_ == lasso_cv.alpha_)[0][0]
best_cv_mse = mean_mse[best_alpha_idx]

# 对应训练集方差
y_var = np.var(y_train)

# 计算交叉验证 R² = 1 - MSE / Var(y)
best_cv_r2 = 1 - best_cv_mse / y_var
print(f"交叉验证 R² (alpha={best_alpha:.6f}): {best_cv_r2:.6f}")

C:\Users\Naoya\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 12.010017277003499, tolerance: 3.6665452820778537
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Naoya\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.161227515545761, tolerance: 3.6665452820778537
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Naoya\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 12.845346792897544, tolerance: 3.6491051741499585
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Naoya\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:68

✅ 最优 alpha: 0.000100
R²:in model: 0.758372
R²:out of model: 0.760958

📊 非零特征数量: 70
     Feature  Coefficient
71      城市_4     2.288241
70      城市_3     1.187294
74      城市_7     0.775178
56  coord_y2     0.615776
26        整租     0.480061
59    环线_二环内     0.302228
13    面积_处理后     0.202296
63    环线_内环内     0.174280
32      集中供暖     0.123510
64  环线_内环至中环     0.107024
交叉验证 R² (alpha=0.000100): 0.757813


C:\Users\Naoya\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.378e+00, tolerance: 4.565e+00
  model = cd_fast.enet_coordinate_descent(


In [35]:
#最佳线性
def best_linear_model(X_train, X_test, y_train, y_test):
    """
    OLS 最佳线性模型训练与验证
    """
    # -------------------------------
    #模型训练
    # -------------------------------
    model = sm.OLS(y_train, X_train)
    results = model.fit()

    # -------------------------------
    #输出模型信息（in-sample）
    # -------------------------------
    print("===== 模型拟合摘要（训练集） =====")
    print(results.summary())

    # in-sample R²
    r2_in = results.rsquared
    print(f"\n训练集（in-sample）R²: {r2_in:.4f}")

    # -------------------------------
    #验证集预测
    # -------------------------------
    y_pred = results.predict(X_test)
    r2_out = r2_score(y_test, y_pred)
    mse_out = mean_squared_error(y_test, y_pred)
    print(f"\n验证集（out-of-sample）R²: {r2_out:.4f}")
    print(f"验证集 MSE: {mse_out:.4f}")

    # -------------------------------
    #输出系数排序
    # -------------------------------
    coef = pd.Series(results.params, index=X_train.columns)
    coef_sorted = coef.sort_values(key=abs, ascending=False)
    print("\n前20个重要特征系数:")
    print(coef_sorted.head(20))

    return results, coef_sorted, {'R2_in': r2_in, 'R2_out': r2_out, 'MSE_out': mse_out}

# -------------------------------
#调用函数
# -------------------------------
results, coef_sorted, metrics = best_linear_model(X_train, X_test, y_train, y_test)

#交叉验证
lr = LinearRegression()
scores = cross_val_score(lr, X_train, y_train, cv=5, scoring='r2')
print("交叉验证 R²:", np.mean(scores))

===== 模型拟合摘要（训练集） =====
                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.764
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     3366.
Date:                Wed, 29 Oct 2025   Prob (F-statistic):               0.00
Time:                        18:09:44   Log-Likelihood:                -33395.
No. Observations:               79119   AIC:                         6.694e+04
Df Residuals:                   79042   BIC:                         6.766e+04
Df Model:                          76                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.4597      0

In [44]:
# 读取已有 submission_price.csv
submission = pd.read_csv('submission_price.csv')

# 对齐列，保证和训练集一致
feature_cols = X_train.columns.drop('const')  # 不含常数项
X_test_new = test_price.reindex(columns=feature_cols, fill_value=0)

# 标准化数值列
X_test_new[num_features] = scaler.transform(X_test_new[num_features])

# 添加常数项
X_test_new = sm.add_constant(X_test_new, has_constant='add')

# 预测 rent
y_pred_log = results.predict(X_test_new)
y_pred = np.exp(y_pred_log)

# 生成新的 DataFrame：rent ID 从 2000000 开始
new_ids = np.arange(2000000, 2000000 + len(y_pred))
new_submission = pd.DataFrame({
    'ID': new_ids,
    'Price': y_pred
})

# 追加到原来的 submission
submission = pd.concat([submission, new_submission], ignore_index=True)

# 输出
submission.to_csv('submission_price_with_rent.csv', index=False)
print("已输出新的 submission 文件，追加了 rent 预测值")

已输出新的 submission 文件，追加了 rent 预测值
